Select this slide when playing slideshow

In [1]:
#Setup
import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<p style="font-family: Montserrat; color: #203443">- What is a subscription?</p>

<p style="font-family: Montserrat; color: #203443">- How can we subscribe to events using the Notifications Application Programming Interface (API) via the LUSID Website?</p>

<p style="font-family: Montserrat; color: #203443">- How can we update a subscription to include a filter on the events of interest using the Notifications API via the Python SDK?</p>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">So what is a subscription?</p>

<p style="font-family: Montserrat; color: #203443">- Rather than being triggered directly off the back of an event, notifications (webhooks, emails, text message etc.) are triggered based on a subscription.</p>

<p style="font-family: Montserrat; color: #203443">- A subscription is linked to a single type of event, you cannot have a subscription linked to multiple event types.</p>

<p style="font-family: Montserrat; color: #203443">- You can create as many subscriptions as you like to a particular event type. You can subscribe every time the event occurs, or filter on event properties to subscribe only in a particular set of circumstances.</p>

<p style="font-family: Montserrat; color: #203443">- You can create as many notifications as you like and attach them to a subscription.</p>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Updating a subscription</p>

In [2]:
import pprint

import lusid_notifications
from lusidjam import RefreshingToken
import fbnsdkutilities.utilities as utils

lusid_api_url = os.environ["FBN_LUSID_API_URL"]
notifications_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "notifications"
os.environ["FBN_NOTIFICATIONS_API_URL"] = notifications_api_url

api_factory = utils.ApiClientFactory(lusid_notifications, token=RefreshingToken())
event_types_api = api_factory.build(lusid_notifications.api.EventTypesApi)
subscriptions_api = api_factory.build(lusid_notifications.api.SubscriptionsApi)

In [3]:
try:
    subscriptions_api.get_subscription(
        scope="FinbourneUniversity",
        code="PortfolioCreated")
    
    subscriptions_api.delete_subscription(
        scope="FinbourneUniversity",
        code="PortfolioCreated"
    )
    
except lusid_notifications.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise
        
subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope="FinbourneUniversity",
            code="PortfolioCreated",
        ),
        display_name="Portfolio Created",
        description="A Portfolio has been Created",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="TransactionPortfolioCreated",
            filter=None
        )
    )
)

{'created_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 340745, tzinfo=tzlocal()),
 'created_by': '00u4edvny74dXgwzK2p7',
 'description': 'A Portfolio has been Created',
 'display_name': 'Portfolio Created',
 'id': {'code': 'PortfolioCreated', 'scope': 'FinbourneUniversity'},
 'last_modified_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 340745, tzinfo=tzlocal()),
 'last_modified_by': '00u4edvny74dXgwzK2p7',
 'matching_pattern': {'event_type': 'TransactionPortfolioCreated',
                      'filter': None},
 'status': 'Active'}

In [4]:
response = subscriptions_api.list_subscriptions(filter="id.scope eq 'FinbourneUniversity'")
pprint.pprint(response)

{'href': 'https://fbn-ci.lusid.com/notifications/api/subscriptions/?filter=id.scope%20eq%20%27FinbourneUniversity%27&sortBy=&limit=2000&page=0AcAACEAaWQuc2NvcGUgZXEgJ0ZpbmJvdXJuZVVuaXZlcnNpdHknAAA%3D',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQDD84SN29U:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': [{'created_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 340745, tzinfo=tzlocal()),
             'created_by': '00u4edvny74dXgwzK2p7',
             'description': 'A Portfolio has been Created',
             'display_name': 'Portfolio Created',
             'id': {'code': 'PortfolioCreated', 'scope': 'FinbourneUniversity'},
             'last_modified_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 340745, tzinfo=tzlocal()),
             'last

In [5]:
event_types = event_types_api.list_event_types().values
pprint.pprint([event_type for event_type in event_types if event_type.id == "TransactionPortfolioCreated"])

[{'application': 'Lusid',
 'description': 'An event of this type is fired whenever a new transaction '
                'portfolio is created in LUSID',
 'entity': None,
 'event_name': 'Transaction Portfolio Created',
 'id': 'TransactionPortfolioCreated',
 'json_schema': {'properties': {'PortfolioCode': {'type': 'string'},
                                'PortfolioCreated': {'format': 'date-time',
                                                     'type': 'string'},
                                'PortfolioDisplayName': {'type': 'string'},
                                'PortfolioScope': {'type': 'string'}},
                 'required': ['PortfolioDisplayName',
                              'PortfolioScope',
                              'PortfolioCode',
                              'PortfolioCreated']}}]


In [6]:
subscription = [sub for sub in response.values if sub.id.code == "PortfolioCreated"][0]

updated_subscription_response = subscriptions_api.update_subscription(
    scope=subscription.id.scope,
    code=subscription.id.code,
    update_subscription=lusid_notifications.UpdateSubscription(
        display_name=subscription.display_name,
        description=subscription.description,
        status=subscription.status,
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type=subscription.matching_pattern.event_type,
            filter="PortfolioDisplayName eq 'GreatPortfolio'"
        )
    )
)

pprint.pprint(updated_subscription_response)


{'created_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 340745, tzinfo=tzlocal()),
 'created_by': '00u4edvny74dXgwzK2p7',
 'description': 'A Portfolio has been Created',
 'display_name': 'Portfolio Created',
 'id': {'code': 'PortfolioCreated', 'scope': 'FinbourneUniversity'},
 'last_modified_at': datetime.datetime(2023, 5, 10, 9, 13, 26, 710654, tzinfo=tzlocal()),
 'last_modified_by': '00u4edvny74dXgwzK2p7',
 'matching_pattern': {'event_type': 'TransactionPortfolioCreated',
                      'filter': "PortfolioDisplayName eq 'GreatPortfolio'"},
 'status': 'Active'}


In [7]:
subscriptions_api.delete_subscription(
    scope=updated_subscription_response.id.scope,
    code=updated_subscription_response.id.code
)

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What have we covered?</p>

<p style="font-family: Montserrat; color: #203443">- Subscriptions are required in order to setup notifications in response to events.</p>

<p style="font-family: Montserrat; color: #203443">- A subscription is only linked to a single event type.</p>

<p style="font-family: Montserrat; color: #203443">- A subscription can subscribe to all events of a particular type. We setup such a subscription using the Notifications API via the LUSID Website.</p>

<p style="font-family: Montserrat; color: #203443">- A subscription can also subscribe to a subset of events of a particulr type using a filter expression. We updated our subscription with such a filter using the Notifications API via the Python SDK.</p>